# Preprocessing + Topic Modeling (Mixed Language Approach)
This notebook serves as a demo for topic modeling GCash App Store Reviews using LDA. The approach of this notebook is to ignore the multi-lingual nature of reviews and move directly to LDA after data cleaning. This notebook will be divided into four parts:

1. Data Cleaning
2. Text Cleaning
3. Modeling (LDA)
4. Exploration of Results

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!ls "/content/drive/My Drive/Data/Reviews/App Store/GCash"

app_info.csv	     gcash_0613-1427.csv	 Old
gcash_0604-2124.csv  gcash_appstore_reviews.csv


In [ ]:
!pip install emot contractions pyLDAvis lda
!python3 -m spacy download en

In [5]:
# importing libraries
import pandas as pd
import numpy as np 
import regex as re
import datetime
import json

from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import contractions


from spacy.lang.tl.stop_words import STOP_WORDS as tl_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

#nltk
import nltk
from nltk.corpus import stopwords

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk
import tqdm

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [297]:
# importing the dataset
df = pd.read_csv('/content/drive/My Drive/Data/Reviews/App Store/GCash/gcash_0613-1427.csv')

In [298]:
df.shape

(4867, 9)

In [299]:
df.dtypes

title                object
date                 object
rating                int64
userName             object
isEdited               bool
review               object
app_name             object
app_id                int64
developerResponse    object
dtype: object

In [300]:
df.head(2)

,title,date,rating,userName,isEdited,review,app_name,app_id,developerResponse
0,It is starting to become pretty annoying,2018-12-29 15:24:59,3,Fhayerie0406,False,Almost everything is smooth except for three t...,gcash,520020791,NaN
1,Actually ok.,2020-05-16 07:33:32,5,i don't usually rate/review,False,This app gets so many negative reviews but it’...,gcash,520020791,NaN


## Part 1: Data Exploration + Cleaning

In [301]:
# Checking for null values
df.isnull().sum() 

title                   0
date                    0
rating                  0
userName                0
isEdited                0
review                  0
app_name                0
app_id                  0
developerResponse    4718
dtype: int64

In [302]:
# Dropping unwanted columns [userName, app_name, app_id, developerResponse]
df = df.drop(['userName', 'app_name', 'app_id', 'developerResponse', 'isEdited'], axis=1)

In [303]:
# Splitting datetime column + Checking date range

# Code block below separated the timestamp [[date]] column into Date and Time column respectively
df['Dates'] = pd.to_datetime(df['date']).dt.date
df['Time'] = pd.to_datetime(df['date']).dt.time
df = df.drop('date', axis=1)

In [304]:
df.sort_values(by='Dates', ascending=False).head(1)

,title,rating,review,Dates,Time
2570,Annoying,2,I can’t open my GCash account 4days after i ac...,2021-06-11,09:05:47


In [305]:
df.sort_values(by='Dates', ascending=True).head(1)

,title,rating,review,Dates,Time
4134,App does not work,1,App does not work. It hangs and automatically ...,2012-05-23,16:24:07


In the section above, we can see that the dataset ranges from 2012-05-23 to 2021-06-11.

In [306]:
df.dtypes

title     object
rating     int64
review    object
Dates     object
Time      object
dtype: object

In [307]:
# Joining the title and review column

df['final_review'] = df.title.map(str) + " " + df.review

In [308]:
df = df.drop(['review','title'], axis=1)

In [309]:
df['word_count'] = df['final_review'].str.split().str.len()

In [310]:
df.head(1)

,rating,Dates,Time,final_review,word_count
0,3,2018-12-29,15:24:59,It is starting to become pretty annoying Almos...,234


In [311]:
df2 = df.copy()

In [312]:
# Filter pandas df for reviews from january 2020 to may 2021
# import datetime
df2 = df2[(df2['Dates']>datetime.date(2020,1,1)) & (df2['Dates']<datetime.date(2021,5,31))] 

In [313]:
df2.shape

(2498, 5)

In [314]:
df2.sort_values(by='word_count')

,rating,Dates,Time,final_review,word_count
4491,1,2021-01-21,02:28:07,Poor WTF,2
4308,5,2020-06-14,14:51:30,omsim 🔥🔥🔥🔥,2
4325,1,2020-04-17,12:20:45,Nakputaka Nakputaka,2
4539,1,2020-09-24,02:47:09,Habducy Hansjaknahd,2
4230,5,2021-05-12,16:22:47,Yes Kaksnsksksk,2
...,...,...,...,...,...
3727,1,2020-01-04,12:59:40,Don’t trust your money with them If there is a...,305
3751,2,2021-04-24,01:39:51,"The best in PH, because it’s the only thing we...",343
3206,1,2020-02-15,11:35:03,Too many issues/Customer Service unhelpful Tho...,349
3778,1,2020-03-18,14:44:27,"App is horrible Hi, this is my 2nd time of sen...",406


In [315]:
df2['word_count'].value_counts().sort_index().head(10)

2     14
3      4
4     22
5     23
6     34
7     31
8     53
9     49
10    46
11    48
Name: word_count, dtype: int64

In [316]:
df2.loc[df2['word_count'].isin([2])]

,rating,Dates,Time,final_review,word_count
3670,1,2021-02-03,01:50:17,-0 Disappointing.,2
3927,5,2021-04-25,03:08:11,Good Good,2
3943,2,2020-09-28,18:43:16,Scammers. Trash.,2
4006,1,2021-01-28,09:19:40,😤 😤,2
4007,1,2021-04-15,10:51:18,. 🤦🏻‍♀️,2
4230,5,2021-05-12,16:22:47,Yes Kaksnsksksk,2
4272,5,2020-12-11,08:23:09,Mae0210 5star,2
4286,1,2020-10-05,15:50:45,trashhhhh trashhhhhh,2
4299,1,2020-07-26,01:10:49,ULOL Kakainis,2
4308,5,2020-06-14,14:51:30,omsim 🔥🔥🔥🔥,2


## Part 2: Text Cleaning

Now that we have done the initial cleaning of the df, let us now proceed to text cleaning for topic modeling purposes.

My approach in this notebook would be as follows:
1. Do the necessary text cleaning steps (remove encodings, lowercase, strip punctutation, expand contractions etc)
2. Remove Filipino stop words and possibly lemmatize Filipino words
3. Remove English stop words and lemmatize
4. Tokenization and other methods

In [317]:
df2.head(4)

,rating,Dates,Time,final_review,word_count
1,5,2020-05-16,07:33:32,Actually ok. This app gets so many negative re...,205
2,3,2020-02-04,00:25:57,Cash In Concern A little bit disappointed when...,186
6,5,2020-06-07,04:57:17,Gcash user since 2018 I've been using gcash fo...,173
7,1,2020-04-26,04:57:47,GCash is Losing its Prime Okay GCash dati 3-4 ...,250


#### Remove emojis and emoticons

In [318]:
# Remove emojis and emoticons

# Code block 1: remove emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Example
remove_emoji("Have fun with NLP! 😃😃")

'Have fun with NLP! '

In [319]:
# apply the remove emoji function to the review column
df2['final_review'] = df2['final_review'].apply(lambda x: remove_emoji(x))

In [320]:
df2.loc[df2['word_count'].isin([1])]

,rating,Dates,Time,final_review,word_count


In [321]:
# Code block 2: remove emoticons

# from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)


In [322]:
#Example
remove_emoticons("I used to play Super Mario games :)")

'I used to play Super Mario games '

In [323]:
# Applying remove emoticons code
df2['final_review'] = df2['final_review'].apply(lambda x: remove_emoticons(x))

In [324]:
df2['word_count'] = df2['final_review'].str.split().str.len()

In [325]:
df2['word_count'].value_counts().sort_index().head(6)

0     1
1     2
2    11
3     5
4    22
5    28
Name: word_count, dtype: int64

In [326]:
df2.loc[df2['word_count'].isin([2])]

,rating,Dates,Time,final_review,word_count
3670,1,2021-02-03,01:50:17,-0 Disappointing.,2
3927,5,2021-04-25,03:08:11,Good Good,2
3943,2,2020-09-28,18:43:16,Scammers. Trash.,2
4230,5,2021-05-12,16:22:47,Yes Kaksnsksksk,2
4272,5,2020-12-11,08:23:09,Mae0210 5star,2
4286,1,2020-10-05,15:50:45,trashhhhh trashhhhhh,2
4299,1,2020-07-26,01:10:49,ULOL Kakainis,2
4325,1,2020-04-17,12:20:45,Nakputaka Nakputaka,2
4463,4,2021-04-12,08:32:31,svdtstbgdvyvg tvtvyvttttt,2
4491,1,2021-01-21,02:28:07,Poor WTF,2


#### Remove \n, \t, \r

In [327]:
df2['final_review'][12]

'Best Finance App! GCash is a big help, indeed. I’ve been using using this app since 2017 and it really helped me to be more financially literate for it has a lot of features that will enable youth professionals to efficiently manage their finances. \r\n\r\nSome of the best features include:\r\n- It allows you to buy load and earn rebates! \r\n- Upon registering to promo, you can earn coupons which you can use on scan to pay features\r\n- allows you to save with high interest, and invest too! \r\n- easily transfer funds to bank with real-time posting and no extra fees! \r\n- Scan to pay is very useful\r\n- Secured and convenient e-wallet\r\n- Wide-range of coverage, a lot of establishments are using GCash already!\r\n- You can help nature by just loading or walking! \r\n- Smooth application\r\n\r\nWhy 4 star not 5 then? \r\nI think there is a need for the following upgrades: \r\n\r\n1. GCredit\r\nAllow users to pay it in installment. Like for 5,000 they can choose of to pay it within 3

In [328]:
# Remove \r \n \t
df2['final_review'] = df2['final_review'].replace(r'\r+|\n+|\t+',' ', regex=True)

In [329]:
df2['final_review'][12]

'Best Finance App! GCash is a big help, indeed. I’ve been using using this app since 2017 and it really helped me to be more financially literate for it has a lot of features that will enable youth professionals to efficiently manage their finances.     Some of the best features include:  - It allows you to buy load and earn rebates!   - Upon registering to promo, you can earn coupons which you can use on scan to pay features  - allows you to save with high interest, and invest too!   - easily transfer funds to bank with real-time posting and no extra fees!   - Scan to pay is very useful  - Secured and convenient e-wallet  - Wide-range of coverage, a lot of establishments are using GCash already!  - You can help nature by just loading or walking!   - Smooth application    Why 4 star not 5 then?   I think there is a need for the following upgrades:     1. GCredit  Allow users to pay it in installment. Like for 5,000 they can choose of to pay it within 3 to 5 months.     Include card for

#### Convert reviews to lowercase

In [330]:
df2['final_review'] = df2['final_review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df2['final_review'].head()

1    actually ok. this app gets so many negative re...
2    cash in concern a little bit disappointed when...
6    gcash user since 2018 i've been using gcash fo...
7    gcash is losing its prime okay gcash dati 3-4 ...
8    read please!! those others who doesn’t like gc...
Name: final_review, dtype: object

#### Removing URLS

In [331]:
# import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

df2['final_review'] = df2.final_review.apply(remove_urls)
df2.final_review.head()

1    actually ok. this app gets so many negative re...
2    cash in concern a little bit disappointed when...
6    gcash user since 2018 i've been using gcash fo...
7    gcash is losing its prime okay gcash dati 3-4 ...
8    read please!! those others who doesn’t like gc...
Name: final_review, dtype: object

#### Expand Contractions

Contractions are words or combinations of words which are shortened by dropping letters and replacing them by an apostrophe. 

Let’s have a look at some examples:
* we’re = we are
* we’ve = we have
* I’d = I would

Note: This step needs to be done before word tokenizer because NLTK word tokenizer has in-built methods for dealing with contractions. However, NLTK word tokenizer's approach separates contractions without expanding. Expanding is a better method than simply separating.

In [332]:
# import contractions

# Example text 
text = ''' She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.'''
  
# creating an empty list 
expanded_words = []     
for word in text.split(): 
  # using contractions.fix to expand the shotened words 
  expanded_words.append(contractions.fix(word))    
    
expanded_text = ' '.join(expanded_words) 
print('Original text: ' + text) 
print('\n') 
print('Expanded_text: ' + expanded_text)

Original text:  She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.


Expanded_text: she will be airport in 30 mins. We are supposed to catch the arrival, are not we? I would love to welcome her personally. it will be an awesome vacation.


In [333]:
df2['final_review'] = df2['final_review'].apply(lambda x: contractions.fix(x))
df.final_review.sample(5)

2431    Errors Fix your apps i cant verify my account ...
1439    Buy load When you buy load IT automatically Lo...
3268    always crashing most of the time especially wh...
4520    Gcash rebates I use gcash as load reloader, bu...
492     Slow Transactions I suddenly experienced slow ...
Name: final_review, dtype: object

#### Removing 1 and 2 word reviews + Separating good and bad reviews into separate dataframes

In [334]:
df2 = df2.reset_index()

In [335]:
df2.final_review[11]

'was good, but... i use gcash to pay my bills. my bank (hsbc, is not one of the banks that has direct communication with gcash, but i was able to cash in via the mastercard/visa option. now they removed it. since hsbc online does not support gcash, i need to physically withdraw money, and cash in via over the counter. this is a shame because of the current pandemic, i want to stay away from people as much as possible. now it is two crowded areas i have to go to. another thing, 7-11’s cash in is a confusing robbery! i did two transactions, because they only allowed up to 10k for a transaction. the 10k cost 60php but the 9.1k transaction was 180php! what?! one more, i linked it with my app store. i bought an in game product, got deducted, but nothing was credited to my account. i contacted the ticket desk, but there has been no reply except for an automated thank you note. convenience was the only thing you had going for you, and you are messing it up just like all other globe services.'

In [336]:
df2['word_count'].value_counts().sort_index().head(6)

0     1
1     2
2    11
3     5
4    22
5    28
Name: word_count, dtype: int64

In [337]:
df2.loc[df2['word_count'].isin([3])]

,index,rating,Dates,Time,final_review,word_count
1416,2715,2,2020-05-01,01:32:09,system always down!!,3
1907,3603,4,2021-01-18,15:06:28,gcash 5 30,3
2061,3872,1,2020-04-21,02:23:58,worst very unreliable,3
2133,3986,1,2020-07-05,04:15:02,no freebies scam,3
2168,4237,1,2021-04-14,16:40:57,stup1d app mandarambong,3


In [338]:
df2['rating'].value_counts()

1    1456
2     373
3     358
5     182
4     129
Name: rating, dtype: int64

In [339]:
df3 = df2.copy()

df3 = df3[~df3['word_count'].isin([0,1,2,3])]

In [340]:
df2.shape

(2498, 6)

In [341]:
df3.shape 

(2479, 6)

In [342]:
df_goodreviews = df3.loc[df['rating'].isin([4,5])].copy()
df_badreviews = df3.loc[df['rating'].isin([1,2,3])].copy()

In [343]:
print(f'df_good_reviews: {df_goodreviews.shape}')
print(f'df_bad_reviews: {df_badreviews.shape}')

df_good_reviews: (555, 6)
df_bad_reviews: (1924, 6)


## Part 3: Modeling

#### Download nltk English stopwords and spacy model (EN for lemmatization)

In [344]:
# Run in python console
# import nltk
nltk.download('stopwords')

# Run in terminal or command prompt
#!python3 -m spacy download |en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Prepare Stop Words (English + Tagalog)

Sources for English stopwords: 
1. nltk library

Sources for Tagalog stopwords:
1. Typical Stop Words: https://github.com/stopwords-iso/stopwords-tl/blob/master/stopwords-tl.json
2. Profanity : https://github.com/jromest/filipino-badwords-list/blob/master/src/filipino-badwords-list.js

In [345]:
# from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [346]:
# import json
with open('/content/drive/My Drive/Data/stopwords/tagalog-sw') as file:
    tl_stopwords = json.load(file)

In [347]:
# Extend stopwords to include tagalog + common gcash related words 
stop_words.extend(tl_stopwords)
stop_words.extend(['gcash','g-cash','po', 'app', 'would','can'])
stop_words.extend(['nyo','naman','yung', 'di', 'wala ','mag', 'nag', 'pag', 'kayo', 'lang', 'ung', 'niyo', 'tapos', 'ba', 'mo', 'please', 'pls'])
stop_words.extend(['nga','yan','yun', 'akong', 'inyo', 'ur', 'star', 'nalang', 'kasi', 'talaga'])
stop_words.extend(['nmn', 'nlng','un','blah', 'tru', 'pwede','like','one','globe', 'guys', 'even', 'seems', 'hi', 'guess'])

#### Tokenize and Clean Up Text some more

In [348]:
# Check columns
df_badreviews.columns

Index(['index', 'rating', 'Dates', 'Time', 'final_review', 'word_count'], dtype='object')

In [349]:
# Convert the review columns into a list
bad_reviews = df_badreviews.final_review.values.tolist()
good_reviews = df_goodreviews.final_review.values.tolist()

In [350]:
# Define a function to tokenize and remove punctuations
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

br_words = list(sent_to_words(bad_reviews))
gr_words = list(sent_to_words(good_reviews))

#### Create Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are *min_count* and *threshold*. The higher the values of these param, the harder it is for words to be combined to bigrams.

###### Bigram and Trigram for good reviews

In [351]:
# Build the bigram and trigram models
g_bigram = gensim.models.Phrases(gr_words, min_count=5, threshold=80) # higher threshold fewer phrases.
g_trigram = gensim.models.Phrases(g_bigram[gr_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
g_bigram_mod = gensim.models.phrases.Phraser(g_bigram)
g_trigram_mod = gensim.models.phrases.Phraser(g_trigram)

# See trigram example
print(g_trigram_mod[g_bigram_mod[gr_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['cash', 'in', 'concern', 'little', 'bit', 'disappointed', 'when', 'cashed', 'in', 'via', 'debit_card', 'waited', 'mins', 'to', 'receive', 'the', 'otp', 'had', 'to', 'restart', 'the', 'application', 'but', 'still', 'same', 'scenario', 'happened', 'when', 'checked', 'my', 'account', 'was', 'already', 'credited', 'by', 'my', 'bank', 'and', 'so', 'reached', 'for', 'customer', 'support', 'since', 'it', 'was', 'not', 'funded', 'in', 'my', 'gcash', 'account', 'the', 'first', 'one', 'talked', 'to', 'said', 'that', 'they', 'had', 'no', 'transactions', 'so', 'need', 'to', 'call', 'my', 'issuing', 'bank', 'called', 'them', 'and', 'said', 'that', 'if', 'it', 'was', 'already', 'deducted', 'on', 'my', 'account', 'meaning', 'it', 'was', 'successful', 'transaction', 'and', 'need', 'to', 'talk', 'to', 'gcash', 'so', 'tried', 'reaching', 'out', 'to', 'gcash', 'customer', 'support', 'but', 'still', 'no', 'update', 'on', 'my', 'concern', 'this', 'was', 'always', 'concern', 'to', 'me', 'whenever', 'cash',

##### Bigram and Trigam for bad reviews

In [352]:
# Build the bigram and trigram models
b_bigram = gensim.models.Phrases(br_words, min_count=5, threshold=80) # higher threshold fewer phrases.
b_trigram = gensim.models.Phrases(b_bigram[br_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
b_bigram_mod = gensim.models.phrases.Phraser(b_bigram)
b_trigram_mod = gensim.models.phrases.Phraser(b_trigram)

# See trigram example
print(b_trigram_mod[b_bigram_mod[br_words[3]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['read', 'please', 'those', 'others', 'who', 'does', 'not', 'like', 'gcash', 'do', 'not', 'expect', 'to', 'much', 'of', 'gcash', 'because', 'not', 'everything', 'you', 'want', 'can', 'give', 'it', 'to', 'those', 'others', 'who', 'have', 'an', 'error', 'maybe', 'give', 'little', 'more', 'patience', 'think', 'of', 'it', 'as', 'the', 'time', 'is', 'giving', 'you', 'challenge', 'like', 'if', 'you', 'can', 'give', 'more', 'patience', 'or', 'not', 'you', 'just', 'really', 'need', 'time', 'and', 'maybe', 'try', 'to', 'solve', 'what', 'is', 'the', 'problem', 'first', 'before', 'you', 'dramatically', 'go', 'crazy', 'anyways', 'other', 'than', 'that', 'base', 'on', 'my', 'experience', 'just', 'do', 'not', 'get', 'why', 'people', 'do', 'not', 'realize', 'that', 'we', 'are', 'all', 'hurting', 'but', 'nobody', 'cares', 'at', 'all', 'maybe', 'at_least', 'try', 'to', 'solve', 'your', 'problems', 'with', 'those', 'people', 'close', 'to', 'you', 'and', 'do', 'not', 'make', 'up', 'things', 'that', 'is',

#### Remove Stopwords, Make Bigrams and Lemmatize (Eng. words only)

##### Functionalizing the steps

I have excluded lemmatization as it brought out errant results when added to tagalog corpus.

In [353]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def g_make_bigrams(texts):
    return [g_bigram_mod[doc] for doc in texts]

def g_make_trigrams(texts):
    return [g_trigram_mod[g_bigram_mod[doc]] for doc in texts]

def b_make_bigrams(texts):
    return [b_bigram_mod[doc] for doc in texts]

def b_make_trigrams(texts):
    return [b_trigram_mod[b_bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

##### Applying the functions to tokenized bad and good reviews


In [354]:
# APPLYING TO TOKENIZED GOOD REVIEWS

# Remove Stop Words
gr_words_nostops = remove_stopwords(gr_words)

# ### REMOVED LEMMATIZATION COS IT AFFECTS THE TAGALOG WORDS / TAGALOG WORDS ARE RENDERED INEFFECTIVE
# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# gr_lemmatized = lemmatization(gr_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


# Form Bigrams
gr_words_bigrams = g_make_bigrams(gr_words_nostops)

# Form Trigrams
gr_words_trigrams = g_make_trigrams(gr_words_bigrams)


In [355]:
gr_words_trigrams[13]

['inferior',
 'functionality',
 'useful',
 'stay',
 'away',
 'new',
 'review',
 'cannot',
 'link',
 'bank',
 'account',
 'system',
 'keeps',
 'rejecting',
 'mobile',
 'banking',
 'password',
 'inputting',
 'correct',
 'got',
 'locked',
 'mobile',
 'banking',
 'account',
 'trying',
 'three',
 'times',
 'many',
 'errors',
 'inconvenient',
 'making',
 'hard',
 'users',
 'got',
 'locked',
 'forgot',
 'mpin',
 'security',
 'questions',
 'forget',
 'mpin',
 'totally',
 'ridiculous',
 'almost',
 'impossible',
 'recover',
 'account',
 'happens',
 'locked',
 'hours',
 'try',
 'making',
 'things',
 'inconvenient',
 'users',
 'cash',
 'function',
 'online',
 'banking',
 'work',
 'careful',
 'receive',
 'money',
 'wallet',
 'credit',
 'card',
 'linking',
 'working',
 'well',
 'useful',
 'mail',
 'support',
 'equally',
 'unhelpful',
 'provides',
 'totally',
 'useless',
 'answers',
 'worthwhile',
 'download',
 'stay',
 'away']

In [356]:
# APPLYING TO TOKENIZED BAD REVIEWS

# Remove Stop Words
br_words_nostops = remove_stopwords(br_words)

# ## REMOVED LEMMATIZATION COS IT AFFECTS THE TAGALOG WORDS / TAGALOG WORDS ARE RENDERED INEFFECTIVE
# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# br_lemmatized = lemmatization(br_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Form Bigrams
br_words_bigrams = b_make_bigrams(br_words_nostops)

br_words_trigrams = b_make_trigrams(br_words_bigrams)




In [357]:
br_words_trigrams[3]


['read',
 'others',
 'expect',
 'much',
 'everything',
 'want',
 'give',
 'others',
 'error',
 'maybe',
 'give',
 'little',
 'patience',
 'think',
 'time',
 'giving',
 'challenge',
 'give',
 'patience',
 'really',
 'need',
 'time',
 'maybe',
 'try',
 'solve',
 'problem',
 'first',
 'dramatically',
 'go',
 'crazy',
 'anyways',
 'base',
 'experience',
 'get',
 'people',
 'realize',
 'hurting',
 'nobody',
 'cares',
 'maybe',
 'least',
 'try',
 'solve',
 'problems',
 'people',
 'close',
 'make',
 'things',
 'true',
 'start',
 'girl',
 'hurting',
 'girls',
 'boys',
 'well',
 'things',
 'said',
 'right',
 'happy',
 'using',
 'bye',
 'anyways',
 'sorry',
 'telling',
 'feelings',
 'cannot',
 'take',
 'anymore']

#### Create the Dictionary and Corpus needed for Topic Modeling

The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

##### Good Reviews -  Dictionary and Corpus

In [358]:
# Create Dictionary
g_id2word = corpora.Dictionary(gr_words_trigrams)
g_id2word.filter_extremes(no_below=10, no_above=0.35)
g_id2word.compactify()
# Create Corpus
g_texts = gr_words_trigrams

# Term Document Frequency
g_corpus = [g_id2word.doc2bow(text) for text in g_texts]

# View
print(g_corpus[:1])

[[(0, 3), (1, 2), (2, 2), (3, 1), (4, 2), (5, 1), (6, 4), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 2), (18, 1), (19, 2), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1)]]


##### Bad Reviews -  Dictionary and Corpus

In [359]:
# Create Dictionary
b_id2word = corpora.Dictionary(br_words_trigrams)
b_id2word.filter_extremes(no_below=10, no_above=0.35)
b_id2word.compactify()

# Create Corpus
b_texts = br_words_trigrams

# Term Document Frequency
b_corpus = [b_id2word.doc2bow(text) for text in b_texts]

# View
print(b_corpus[:1])

[[(0, 1), (1, 2), (2, 2), (3, 4), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 2)]]


#### Building the Topic Model

##### Optizing Number of Topics for Good Reviews

In [ ]:
# Build initial LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=g_corpus,
                                           id2word=g_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [361]:
print(json.dumps(lda_model.print_topics(), indent=1))

[
 [
  0,
  "0.067*\"mpin\" + 0.059*\"open\" + 0.043*\"wrong\" + 0.038*\"login\" + 0.037*\"keeps\" + 0.036*\"always\" + 0.030*\"limit\" + 0.026*\"crashing\" + 0.025*\"still\" + 0.024*\"tried\""
 ],
 [
  1,
  "0.159*\"cash\" + 0.082*\"load\" + 0.063*\"transaction\" + 0.043*\"new\" + 0.037*\"never\" + 0.036*\"paypal\" + 0.031*\"unavailable\" + 0.029*\"days\" + 0.028*\"bank\" + 0.028*\"month\""
 ],
 [
  2,
  "0.102*\"cannot\" + 0.071*\"account\" + 0.050*\"update\" + 0.046*\"fix\" + 0.036*\"need\" + 0.036*\"id\" + 0.029*\"verify\" + 0.021*\"verification\" + 0.021*\"use\" + 0.020*\"time\""
 ],
 [
  3,
  "0.067*\"money\" + 0.052*\"number\" + 0.041*\"get\" + 0.033*\"email\" + 0.026*\"code\" + 0.025*\"account\" + 0.024*\"sent\" + 0.024*\"verified\" + 0.023*\"ticket\" + 0.021*\"phone\""
 ],
 [
  4,
  "0.140*\"service\" + 0.101*\"code\" + 0.064*\"bad\" + 0.060*\"add\" + 0.050*\"support\" + 0.038*\"log\" + 0.037*\"tried\" + 0.035*\"needs\" + 0.035*\"transactions\" + 0.031*\"everything\""
 ],
 [
 

In [362]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=gr_words_trigrams, dictionary=g_id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [363]:
# #Optimizing topic models

# import numpy as np
# import tqdm
# grid = {}
# grid['Validation_Set'] = {}
# # Topics range
# min_topics = 2
# max_topics = 8
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
# # Alpha parameter
# alpha = list(np.arange(0.01, 1, 0.3))
# alpha.append('symmetric')
# alpha.append('asymmetric')
# # Beta parameter
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')
# # Validation sets
# num_of_docs = len(g_corpus)
# corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
#                # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#                #gensim.utils.ClippedCorpus(g_corpus, num_of_docs*0.75), 
#                g_corpus]
# corpus_title = ['75% Corpus', '100% Corpus']
# model_results = {'Validation_Set': [],
#                  'Topics': [],
#                  'Alpha': [],
#                  'Beta': [],
#                  'Coherence': []
#                 }
# # Can take a long time to run
# if 1 == 1:
#     pbar = tqdm.tqdm(total=540)
    
#     # iterate through validation corpuses
#     for i in range(len(corpus_sets)):
#         # iterate through number of topics
#         for k in topics_range:
#             # iterate through alpha values
#             for a in alpha:
#                 # iterare through beta values
#                 for b in beta:
#                     # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=g_id2word, 
#                                                   k=k, a=a, b=b)
#                     # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
                    
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv('gcash_gr_lda_tuning_results.csv', index=False)
#     pbar.close()

In [364]:
# model_cv_df_gr = pd.DataFrame(model_results) 
# model_cv_df_gr.sort_values(by='Coherence',ascending=False)

In [ ]:
# Build optimized LDA model

lda_model_experimental = gensim.models.LdaMulticore(corpus=g_corpus,
                                           id2word=g_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

In [366]:
print(json.dumps(lda_model_experimental.print_topics(), indent=1))

[
 [
  0,
  "0.097*\"money\" + 0.050*\"verification\" + 0.028*\"email\" + 0.026*\"ticket\" + 0.024*\"student_id\" + 0.024*\"send\" + 0.022*\"support\" + 0.021*\"login\" + 0.021*\"get\" + 0.018*\"cannot\""
 ],
 [
  1,
  "0.073*\"cash\" + 0.037*\"transaction\" + 0.035*\"load\" + 0.025*\"bank\" + 0.018*\"card\" + 0.017*\"way\" + 0.016*\"hope\" + 0.016*\"convenient\" + 0.016*\"already\" + 0.015*\"days\""
 ],
 [
  2,
  "0.075*\"cannot\" + 0.070*\"update\" + 0.047*\"account\" + 0.046*\"fix\" + 0.036*\"need\" + 0.033*\"working\" + 0.025*\"updated\" + 0.025*\"verify\" + 0.021*\"gcredit\" + 0.019*\"access\""
 ],
 [
  3,
  "0.061*\"always\" + 0.054*\"time\" + 0.041*\"cannot\" + 0.034*\"error\" + 0.022*\"every\" + 0.021*\"want\" + 0.018*\"try\" + 0.016*\"annoying\" + 0.015*\"using\" + 0.015*\"keeps\""
 ],
 [
  4,
  "0.056*\"code\" + 0.053*\"number\" + 0.037*\"cannot\" + 0.035*\"tried\" + 0.034*\"mpin\" + 0.032*\"wrong\" + 0.031*\"account\" + 0.028*\"open\" + 0.026*\"new\" + 0.024*\"log\""
 ],
 [


##### Optimizing Number of Topics for Bad Reviews

In [ ]:
# Build LDA model
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=b_corpus,
                                           id2word=b_id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



In [368]:
print(json.dumps(lda_model2.print_topics(), indent=1))

[
 [
  0,
  "0.103*\"error\" + 0.074*\"log\" + 0.050*\"system\" + 0.048*\"easy\" + 0.043*\"gcredit\" + 0.030*\"anymore\" + 0.023*\"unreliable\" + 0.023*\"trash\" + 0.023*\"useful\" + 0.022*\"rebates\""
 ],
 [
  1,
  "0.121*\"cash\" + 0.081*\"bank\" + 0.058*\"transaction\" + 0.043*\"mpin\" + 0.041*\"wala\" + 0.029*\"transfer\" + 0.024*\"linked\" + 0.020*\"link\" + 0.020*\"paymaya\" + 0.020*\"paypal\""
 ],
 [
  2,
  "0.087*\"always\" + 0.072*\"service\" + 0.046*\"thing\" + 0.035*\"login\" + 0.033*\"everytime\" + 0.033*\"something\" + 0.032*\"convenient\" + 0.029*\"however\" + 0.026*\"user\" + 0.024*\"services\""
 ],
 [
  3,
  "0.102*\"cannot\" + 0.053*\"fix\" + 0.043*\"update\" + 0.036*\"use\" + 0.027*\"using\" + 0.026*\"issue\" + 0.024*\"time\" + 0.023*\"try\" + 0.022*\"problem\" + 0.021*\"still\""
 ],
 [
  4,
  "0.067*\"account\" + 0.059*\"money\" + 0.036*\"id\" + 0.034*\"verification\" + 0.027*\"get\" + 0.026*\"verify\" + 0.025*\"already\" + 0.024*\"email\" + 0.020*\"customer_service\

In [369]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=gr_words_trigrams, dictionary=g_id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

#### Visualizing LDA model results

##### Plotting topic models using pyLDAvis

In [370]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, g_corpus, g_id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.215806  0.087955       1        1  33.764828
3      0.034671 -0.022791       2        1  21.489895
5     -0.141942 -0.005518       3        1  19.419991
0      0.180864  0.046760       4        1  10.929935
1     -0.061159 -0.309536       5        1   8.862454
4     -0.228240  0.203130       6        1   5.532896, topic_info=             Term        Freq       Total Category  logprob  loglift
6            cash   87.000000   87.000000  Default  30.0000  30.0000
221          code   69.000000   69.000000  Default  29.0000  29.0000
40        service   48.000000   48.000000  Default  28.0000  28.0000
86         cannot  240.000000  240.000000  Default  27.0000  27.0000
74           load   45.000000   45.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
114  verification    6.444504   66.177944   Topic6  -3.9673   0.5653
174            us    2.186200   25.031233   Topic6  -5.0484   0.4565
208          wala    1.906415   23.028758   Topic6  -5.1853   0.4029
44           wait    1.208746   18.253701   Topic6  -5.6410   0.1797
120          make    1.473866   29.063913   Topic6  -5.4427  -0.0871

[259 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
67        2  0.342418     able
67        6  0.616352     able
220       2  0.940739      acc
100       1  0.979237   access
0         1  0.800705  account
...     ...       ...      ...
46        2  0.302113    wrong
46        4  0.673944    wrong
184       3  0.520241      yet
184       5  0.074320      yet
184       6  0.371601      yet

[328 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 6, 1, 2, 5])

In [371]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model2, b_corpus, b_id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.221751 -0.122820       1        1  31.944996
3     -0.234072 -0.117288       2        1  23.963684
5     -0.078587  0.252804       3        1  17.756355
1      0.280161 -0.260119       4        1  11.587268
2      0.173353  0.195088       5        1   8.098897
0      0.080897  0.052335       6        1   6.648799, topic_info=        Term        Freq       Total Category  logprob  loglift
52    cannot  600.000000  600.000000  Default  30.0000  30.0000
9       cash  336.000000  336.000000  Default  29.0000  29.0000
588     code  346.000000  346.000000  Default  28.0000  28.0000
1    account  514.000000  514.000000  Default  27.0000  27.0000
68     money  455.000000  455.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
180   invest   21.055202   21.913306   Topic6  -4.3253   2.6708
574   failed   20.356863   21.213870   Topic6  -4.3591   2.6695
177     fees   19.161799   20.018548   Topic6  -4.4196   2.6670
188   smooth   18.349924   19.209171   Topic6  -4.4629   2.6650
80       use   29.130838  278.390834   Topic6  -4.0007   0.4535

[239 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         3  0.969805      able
0         6  0.012595      able
389       2  0.989898    access
1         1  0.997609   account
86        4  0.978784  accounts
...     ...       ...       ...
537       6  0.948225     worse
230       1  0.989360     worst
48        2  0.995368     wrong
589       3  0.982275       wtf
85        1  0.989306       yet

[241 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 6, 2, 3, 1])

##### Checking time series distribution of reviews [ January 1 2020 to May 31 2021 ]